In [ ]:
# Bài thực hành: Phát hiện đối tượng với YOLOv8
# -------------------------------------------



In [ ]:
# Phần 1: Cài đặt và chuẩn bị môi trường
# -------------------------------------------
# Cài đặt thư viện Ultralytics YOLOv8
!pip install ultralytics

# Cài đặt các thư viện cần thiết khác
!pip install opencv-python matplotlib

In [ ]:
# Import các thư viện
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
from google.colab import drive
from IPython.display import display, Image

In [ ]:
# Mount Google Drive (nếu cần lưu kết quả)
drive.mount('/content/drive')

# Tạo thư mục để lưu kết quả
!mkdir -p /content/results

In [ ]:
# Phần 2: Tải và chuẩn bị mô hình
# -------------------------------------------

# Tải mô hình YOLOv8 đã được huấn luyện trước
model = YOLO('yolov8n.pt')  # sử dụng mô hình YOLOv8 nano

# Hiển thị thông tin về mô hình
print(f"Mô hình đã tải: YOLOv8n")
print(f"Các lớp có thể nhận dạng: {model.names}")

In [ ]:
# Phần 3: Thực hiện phát hiện đối tượng trên bộ dữ liệu COCO
# -------------------------------------------

# Tải một số ảnh mẫu từ bộ dữ liệu COCO
!mkdir -p /content/images
!wget -q -O /content/images/bus.jpg https://ultralytics.com/images/bus.jpg
!wget -q -O /content/images/zidane.jpg https://ultralytics.com/images/zidane.jpg

In [ ]:
# Kiểm tra xem ảnh đã tải thành công chưa
!ls -la /content/images/

In [ ]:
# Hiển thị ảnh gốc
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
img_bus = cv2.imread('/content/images/bus.jpg')
if img_bus is not None:
    img_bus = cv2.cvtColor(img_bus, cv2.COLOR_BGR2RGB)
    plt.imshow(img_bus)
    plt.title('Ảnh gốc 1: Bus')
    plt.axis('off')
else:
    print("Không thể đọc ảnh bus.jpg")

plt.subplot(1, 2, 2)
img_zidane = cv2.imread('/content/images/zidane.jpg')
if img_zidane is not None:
    img_zidane = cv2.cvtColor(img_zidane, cv2.COLOR_BGR2RGB)
    plt.imshow(img_zidane)
    plt.title('Ảnh gốc 2: Người')
    plt.axis('off')
else:
    print("Không thể đọc ảnh zidane.jpg")

plt.tight_layout()
plt.show()

In [ ]:
# Thực hiện phát hiện đối tượng
if os.path.exists('/content/images/bus.jpg') and os.path.exists('/content/images/zidane.jpg'):
    results_bus = model('/content/images/bus.jpg')
    results_zidane = model('/content/images/zidane.jpg')

    # Hiển thị kết quả phát hiện
    plt.figure(figsize=(12, 6))

    # Ảnh kết quả 1
    plt.subplot(1, 2, 1)
    res_plotted_bus = results_bus[0].plot()
    plt.imshow(cv2.cvtColor(res_plotted_bus, cv2.COLOR_BGR2RGB))
    plt.title('Kết quả phát hiện: Bus')
    plt.axis('off')

    # Ảnh kết quả 2
    plt.subplot(1, 2, 2)
    res_plotted_zidane = results_zidane[0].plot()
    plt.imshow(cv2.cvtColor(res_plotted_zidane, cv2.COLOR_BGR2RGB))
    plt.title('Kết quả phát hiện: Người')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

      # Phần 4: Phân tích kết quả từ bộ dữ liệu COCO
    # -------------------------------------------

    # Hiển thị thông tin chi tiết về các đối tượng được phát hiện
    print("Chi tiết đối tượng phát hiện trong ảnh Bus:")
    for i, result in enumerate(results_bus):
        boxes = result.boxes
        print(f"Số lượng đối tượng phát hiện: {len(boxes)}")

        for j, box in enumerate(boxes):
            class_id = int(box.cls.item())
            confidence = box.conf.item()
            coordinates = box.xyxy.tolist()[0]  # [x1, y1, x2, y2]

            print(f"Đối tượng {j+1}:")
            print(f"  - Lớp: {model.names[class_id]}")
            print(f"  - Độ tin cậy: {confidence:.2f}")
            print(f"  - Tọa độ (x1, y1, x2, y2): [{int(coordinates[0])}, {int(coordinates[1])}, {int(coordinates[2])}, {int(coordinates[3])}]")
else:
    print("Không tìm thấy ảnh mẫu. Vui lòng kiểm tra lại đường dẫn.")

In [ ]:
# Phần 5: Thử nghiệm với các ngưỡng tin cậy khác nhau
# -------------------------------------------

if os.path.exists('/content/images/bus.jpg'):
    # Thiết lập ngưỡng tin cậy khác nhau và xem kết quả
    confidence_thresholds = [0.25, 0.5, 0.75]

    plt.figure(figsize=(15, 10))
    for i, conf in enumerate(confidence_thresholds):
        # Thực hiện phát hiện với ngưỡng tùy chỉnh
        results = model('/content/images/bus.jpg', conf=conf)

        # Vẽ kết quả
        plt.subplot(1, 3, i+1)
        res_plotted = results[0].plot()
        plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
        plt.title(f'Ngưỡng tin cậy: {conf}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("Không tìm thấy ảnh bus.jpg")

In [ ]:
# Phần 6: Đánh giá hiệu suất của mô hình
# -------------------------------------------
# So sánh với các biến thể YOLOv8 khác
performance_data = {
    'Model': ['YOLOv8n', 'YOLOv8s', 'YOLOv8m', 'YOLOv8l', 'YOLOv8x'],
    'Size (MB)': [6.7, 21.2, 52.2, 86.0, 130.5],
    'mAP50-95 (%)': [37.3, 44.9, 50.2, 52.9, 53.9],
    'Speed (ms/image)': [8, 10, 16, 22, 28]
}

print("\nSo sánh các biến thể của YOLOv8:")
for i in range(len(performance_data['Model'])):
    print(f"{performance_data['Model'][i]}: "
          f"Size={performance_data['Size (MB)'][i]} MB, "
          f"mAP={performance_data['mAP50-95 (%)'][i]}%, "
          f"Speed={performance_data['Speed (ms/image)'][i]} ms/image")
